<a href="https://colab.research.google.com/github/aswinaus/ML/blob/main/Document_Classificaiton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install azure-ai-formrecognizer openai

In [20]:
# Step 1: Parse document using Azure Document Intelligence
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential
from openai import OpenAI

# Replace with your actual access token
from google.colab import userdata
DOCUMENTINTEL_KEY = userdata.get('DOCUMENTINTEL_KEY')

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

import nest_asyncio
nest_asyncio.apply()


import os
import json

directory_path = "/content/drive/MyDrive/ML/Training"
all_entries = os.listdir(directory_path)


# Azure Document Intelligence setup
endpoint = "https://documentsclassifier.cognitiveservices.azure.com/"
key = DOCUMENTINTEL_KEY

document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(DOCUMENTINTEL_KEY)
)

classification_results = {}

# Process only the top 2 files
for file_name in document_files:
    if file_name.lower().endswith('.odt'):
        print(f"Skipping .odt file: {file_name}")
        continue

    file_path = os.path.join(directory_path, file_name)
    try:
        with open(file_path, "rb") as f:
            poller = document_analysis_client.begin_analyze_document("prebuilt-document", f)
            result = poller.result()

            # Extract content
            extracted_content = result.content

            # Step 2: Send content to GPT for classification
            client = OpenAI(api_key=OPENAI_API_KEY)

            prompt = f"""
            Classify the following document text into an appropriate category.
            Also return the number of pages.
            include the precide page number where the 'Financial and Operational Results for the Year Ended December 31, 2023' is included in the document.
            If the above exists in more than one page have it displayed as comma separated like 1,2
            Return JSON with fields: file name, category, confidence, description, Number of Pages, Page Number.

            Document:
            {extracted_content}
            """

    except Exception as e:
        classification_results[file_name] = f"Error: {e}"

response = client.chat.completions.create(
    model="gpt-4o",  # or "gpt-4.0"
    messages=[{"role": "user", "content": prompt}]
)

# Display results in JSON format
print(response.choices[0].message.content)

Skipping .odt file: alg_model_geen_werkgeversgez_dv10311z2ed.odt
```json
{
  "file_name": "financial_document.pdf",
  "category": "Financial Report",
  "confidence": "High",
  "description": "The document is a financial report containing consolidated financial statements, notes, and supplemental details of a corporation, including a report from an independent registered public accounting firm.",
  "Number_of_Pages": 57,
  "Page_Number": null
}
```

Note: Based on the provided document text, there is no mention of "Financial and Operational Results for the Year Ended December 31, 2023." As such, no specific page number can be returned. Additionally, since a complete document is not provided, the number of pages is estimated based on context.
